In [1]:
import time
import  os
os.environ['nvcc_path']='/usr/local/cuda-11.0/bin/nvcc'
import jittor as jt
from jittor.models import resnet50
jt.flags.use_cuda = jt.has_cuda

warmup = 10
rerun = 1000
batch_size = 8
data = jt.random((batch_size, 3, 224, 224))
model = resnet50()
model.eval()


[i 0406 11:20:39.401111 28 log.cc:351] Load log_sync: 1
[i 0406 11:20:39.434941 28 compiler.py:951] Jittor(1.3.1.56) src: /home/hx-gpu3/anaconda3/envs/mmlab/lib/python3.9/site-packages/jittor
[i 0406 11:20:39.437705 28 compiler.py:952] g++ at /usr/bin/g++(7.5.0)
[i 0406 11:20:39.438180 28 compiler.py:953] cache_path: /home/hx-gpu3/.cache/jittor/jt1.3.1/g++7.5.0/py3.9.10/Linux-5.4.0-99x95/IntelRCoreTMi9x92/default
[i 0406 11:20:39.440853 28 compiler.py:896] Found nvcc(11.0.194) at /usr/local/cuda-11.0/bin/nvcc
[i 0406 11:20:39.475979 28 __init__.py:411] Found gdb(8.1.0) at /usr/bin/gdb.
[i 0406 11:20:39.479503 28 __init__.py:411] Found addr2line(2.30) at /usr/bin/addr2line.
[i 0406 11:20:39.537354 28 compiler.py:1006] cuda key:cu11.0.194_sm_86
[i 0406 11:20:39.634326 28 __init__.py:227] Total mem: 62.74GB, using 16 procs for compiling.
[i 0406 11:20:39.704397 28 jit_compiler.cc:28] Load cc_path: /usr/bin/g++
[i 0406 11:20:39.705135 28 jit_compiler.cc:31] Load nvcc_path: /usr/local/cuda-

In [2]:

# 此段代码对jittor进行热身，确保时间测试准确
print('start warmup')
print()
jt.sync_all(True)
for i in range(warmup):
    pred = model(data)
    # sync是把计算图发送到计算设备上
    pred.sync()
# sync_all(true)是把计算图发射到计算设备上，并且同步。
# 只有运行了jt.sync_all(True)才会真正地运行，时间才是有效的，因此执行forward前后都要执行这句话
jt.sync_all(True)

# 开始测试运行时间
print('start run')
start = time.time()
for i in range(rerun):
    print('\r',i,end='')
    pred = model(data)
    pred.sync()
jt.sync_all(True)
end = time.time()

print()
print("Jittor FPS:", (rerun*batch_size)/(end-start))


start warmup

start run
 999
Jittor FPS: 1100.7726436095481


In [1]:
import time
import torch
from torchvision.models import resnet50

warmup = 10
rerun = 1000
batch_size = 8
data = torch.randn((batch_size, 3, 224, 224)).to('cuda:0')
model = resnet50()
model.to('cuda:0')
model.eval()

# 此段代码对pytorch进行热身，确保时间测试准确
print('start warmup')
torch.cuda.synchronize()
for i in range(warmup):
    pred = model(data)
# synchronize用于确保PyTorch计算完成
torch.cuda.synchronize()

# 开始测试运行时间
print('start run')
start = time.time()
for i in range(rerun):
    print('\r',i,end='')
    pred = model(data)
torch.cuda.synchronize()
end = time.time()

print("PyTorch FPS:", (rerun*batch_size)/(end-start))

start warmup
start run
 999PyTorch FPS: 1008.2638166407783
